# MLOps Workshop: Umgebungseinrichtung

## Einführung
Willkommen zum MLOps Workshop! In diesem Notebook richten wir unsere Entwicklungsumgebung ein und installieren alle notwendigen Tools für unser Customer Churn Prediction Projekt.

## Lernziele
Nach Abschluss dieses Notebooks werden Sie:
- Eine funktionierende Python-Umgebung mit allen erforderlichen Paketen haben
- Die grundlegende Projektstruktur verstehen
- Ein Git-Repository initialisiert haben
- Alle MLOps-Tools auf ihre Funktionsfähigkeit überprüft haben

## 1. Umgebungseinrichtung

### 1.1 Virtuelle Umgebung erstellen
Führen Sie zunächst diese Befehle in Ihrem  aus:

```bash
python -m venv mlops-venv
# Unter Windows
.\mlops-venv\Scripts\activate
# Unter Unix/MacOS
source mlops-venv/bin/activate
```

### 1.2 Abhängigkeiten installieren
Installieren wir die benötigten Pakete:

In [2]:
# Zelle 1: Benötigte Pakete installieren
import sys
!{sys.executable} -m pip install numpy pandas scikit-learn mlflow pytest fastapi uvicorn great-expectations docker python-dotenv matplotlib seaborn


  Using cached mlflow-3.9.0-py3-none-any.whl.metadata (31 kB)
  Using cached pytest-9.0.2-py3-none-any.whl.metadata (7.6 kB)
  Using cached fastapi-0.128.3-py3-none-any.whl.metadata (30 kB)
  Using cached uvicorn-0.40.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
     ---------------------------------------- 0.0/52.8 kB ? eta -:--:--
     ---------------------------------------- 52.8/52.8 kB 2.8 MB/s eta 0:00:00
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 61.0/61.0 kB 3.2 MB/s eta 0:00:00
  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached mlflow_skinny-3.9.0-py3-none-

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [6]:
import sys, subprocess
from pathlib import Path

req = subprocess.check_output([sys.executable, "-m", "pip", "freeze"], text=True)
Path("../requirements.txt").write_text(req, encoding="utf-8")
print("saved to project root")


saved to project root


### 1.3 Installation überprüfen
Führen Sie diese Zelle aus, um zu überprüfen, ob alle Pakete korrekt installiert wurden:

In [1]:
# Zelle 2: Import und Versionscheck
import sys
import numpy as np
import pandas as pd
import mlflow
import great_expectations as ge
from fastapi import FastAPI
import pytest

# Versionen ausgeben
print(f"Python Version: {sys.version}")
print(f"NumPy Version: {np.__version__}")
print(f"Pandas Version: {pd.__version__}")
print(f"MLflow Version: {mlflow.__version__}")
print(f"Great Expectations Version: {ge.__version__}")

c:\Users\bjoer\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Python Version: 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
NumPy Version: 2.4.2
Pandas Version: 2.3.3
MLflow Version: 3.9.0
Great Expectations Version: 1.11.3


In [ ]:
# Datensatz herunterladen
import pandas as pd

url = "https://raw.githubusercontent.com/hansbjoern93/loan-mlops/refs/heads/main/data/raw/loan_data.csv"
df = pd.read_csv(url)
df.to_csv('../data/raw/loan_data.csv', index=False)
print("Datensatz wurde heruntergeladen und in data/raw/loan_data.csv gespeichert")

Datensatz wurde heruntergeladen und in data/raw/loan_data.csv gespeichert


## 5. Einrichtung überprüfen
Führen Sie diese abschließende Überprüfung durch:

In [14]:
# Zelle 6: Abschließende Verifizierung
import os

def verify_setup():
    checks = {
        "Projektstruktur": all(os.path.exists(d) for d in ['../data/raw', '../notebooks', '../src', '../tests']),
        "Git initialisiert": os.path.exists('../.git'),
        "Datensatz heruntergeladen": os.path.exists('../data/raw/loan_data.csv'),
        "Git ignore erstellt": os.path.exists('../.gitignore')
    }
    
    all_passed = True
    for check, passed in checks.items():
        status = "✅" if passed else "❌"
        print(f"{status} {check}")
        all_passed = all_passed and passed
    
    return all_passed

success = verify_setup()
if success:
    print("\nEinrichtung erfolgreich abgeschlossen! Sie können mit dem Workshop beginnen!")
else:
    print("\nEinige Überprüfungen sind fehlgeschlagen. Bitte überprüfen Sie die obige Ausgabe und beheben Sie eventuelle Probleme.")

✅ Projektstruktur
✅ Git initialisiert
✅ Datensatz heruntergeladen
✅ Git ignore erstellt

Einrichtung erfolgreich abgeschlossen! Sie können mit dem Workshop beginnen!
